In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from config import user, password, db_name

### Extract

In [2]:
# Import CSV with AirBnB data
csv_file = "Resources/listings.csv"
airbnb_data_df = pd.read_csv(csv_file)
airbnb_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,2019-12-15,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2020-02-14,0.61,5,168


In [3]:
# Import CSV with community area numbers
csv_file = "Resources/chicago-community-areas.csv"
communities_df = pd.read_csv(csv_file)
communities_df.head()

,Community Area,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,name,Rogers Park,West Ridge,Uptown,Lincoln Square,North Center,Lake View,Lincoln Park,Near North Side,Edison Park,...,Englewood,Greater Grand Crossing,Ashburn,Auburn Gresham,Beverly,Washington Heights,Mount Greenwood,Morgan Park,O'Hare,Edgewater


### Transform

In [4]:
airbnb_data_df = airbnb_data_df.drop(["neighbourhood_group"], axis=1)

In [5]:
airbnb_data_df.set_index("id", inplace=True)

In [30]:
airbnb_data_df.dropna(how = "any", inplace=True)
airbnb_data_df.index.max()

45465696

In [7]:
#listing_columns = ["host_id", "host_name", "minimum nights"]
listing_info_df = airbnb_data_df.iloc[:, [1, 2, 6, 7, 8, 9, 10, 11, 12, 13]]

#df.columns
#df.iloc[:, 2]
location_info_df = airbnb_data_df.iloc[:, [0, 3, 4, 5]]

In [8]:
location_info_df = location_info_df.rename(columns={"name": "address"})

In [29]:
listing_info_df.count
#location_info_df

<bound method DataFrame.count of             host_id         host_name        room_type  price  minimum_nights  \
id                                                                              
2384           2613           Rebecca     Private room     60               2   
4505           5775  Craig & Kathleen  Entire home/apt    105               2   
7126          17928             Sarah  Entire home/apt     60               2   
9811          33004       At Home Inn  Entire home/apt     65               4   
10610          2140              Lois     Private room     21               1   
...             ...               ...              ...    ...             ...   
45351578   77382816            Edmund  Entire home/apt     67               1   
45368527  128265803            Victor     Private room     24               1   
45386114  324740940            Andrea  Entire home/apt     97               1   
45433310  111872967             Julie     Private room     54               

In [10]:
communities_df.columns = communities_df.columns.str.lower()

# Replace spaces in column names with underscores
communities_df.columns = communities_df.columns.str.replace(' ', '_')

communities_df.set_index("community_area", inplace=True)

In [11]:
communities_df = communities_df.transpose()

In [12]:
communities_df = communities_df.rename(columns={"name": "neighborhood"})

In [13]:
communities_df.index = communities_df.index.astype(int)

In [33]:
communities_df.nunique()

community_area
neighborhood    77
dtype: int64

In [14]:
# Import CSV with Chicago 2013 population numbers
csv_file = "Resources/chicago_population2013.csv"
population_df = pd.read_csv(csv_file, sep=';')

In [15]:
# Make the column names lowercase
population_df.columns = population_df.columns.str.lower()

# Replace spaces in column names with underscores
population_df.columns = population_df.columns.str.replace(' ', '_')

In [16]:
population_df.set_index("community_area", inplace=True)
population_df = population_df.transpose()

In [17]:
population_df.head()

community_area,name,population,income,latinos,blacks,white,asian,other
1,Rogers Park,54991,39482,0.244,0.263,0.393,0.064,0.036
2,West Ridge,71942,47323,0.204,0.111,0.427,0.225,0.032
3,Uptown,56362,40324,0.142,0.2,0.516,0.114,0.028
4,Lincoln Square,39493,57749,0.191,0.038,0.631,0.111,0.029
5,North Center,31867,81524,0.136,0.023,0.773,0.045,0.022


In [18]:
# This code does not run by itself: this is sample code

# Make the column names lowercase
df.columns = df.columns.str.lower()

# Replace spaces in column names with underscores
df.columns = df.columns.str.replace(' ', '_')

# Rename Columns
df = df.rename(columns={"OLD NAME": "new_name", "OLD NAME2": "new_name_2"})

# Drop duplicates in column before turning it into the index
df.drop_duplicates("column", inplace=True)
df.set_index("column", inplace=True)

# Choose columns to retain
preferred_columns = ["keep", "these", "columns"]
df_transformed = df[preferred_columns].copy()




NameError: name 'df' is not defined

### Load

In [19]:
# Create database connection
rds_connection_string = f"{user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [20]:
# Check for tables
inspector = inspect(engine)
print(inspector.get_table_names())

['communities']


In [27]:
# Load CSV converted DataFrame into database
listing_info_df.to_sql(name='listing', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "id" of relation "listing" violates not-null constraint
DETAIL:  Failing row contains (null, 2613, Rebecca, Private room, $60.00, 2, 178, 2019-12-15, 2.56, 1, 353).

[SQL: INSERT INTO listing (host_id, host_name, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365) VALUES (%(host_id)s, %(host_name)s, %(room_type)s, %(price)s, %(minimum_nights)s, %(number_of_reviews)s, %(last_review)s, %(reviews_per_month)s, %(calculated_host_listings_count)s, %(availability_365)s)]
[parameters: ({'host_id': 2613, 'host_name': 'Rebecca', 'room_type': 'Private room', 'price': 60, 'minimum_nights': 2, 'number_of_reviews': 178, 'last_review': '2019-12-15', 'reviews_per_month': 2.56, 'calculated_host_listings_count': 1, 'availability_365': 353}, {'host_id': 5775, 'host_name': 'Craig & Kathleen', 'room_type': 'Entire home/apt', 'price': 105, 'minimum_nights

In [ ]:
# Load CSV converted DataFrame into database
df.to_sql(name='table_name', con=engine, if_exists='append', index=False)

In [ ]:
# Confirm data has been added
pd.read_sql_query('select * from table_name', con=engine).head()